In [1]:
import numpy as np
import pandas as pd
from Branch import Branch
from sklearn.ensemble import RandomForestClassifier
from ConjunctionSet import ConjunctionSet
from sklearn.tree import export_graphviz
import pydot
import matplotlib.pyplot as plt
from DataPreperation import *
from ReadDatasetFunctions import *
%matplotlib inline

In [2]:
data,x_columns,y_column = read_tic_tac_toe_dataset()

In [4]:
results=[]
for i in range(10):
    train_x,train_y,test_x,test_y=divide_to_train_test(data,x_columns,y_column )
    rf=RandomForestClassifier(n_estimators=5,max_depth=3)
    rf.fit(train_x,train_y)
    cs=ConjunctionSet(x_columns,rf)
    rf_predicions=rf.predict(test_x)
    conjunctionPredictions=cs.predict(test_x)
    disagreements=np.sum(conjunctionPredictions!=rf_predicions)
    results.append(disagreements)
    if disagreements:
        break

Iteration 1: 8 conjunctions
Iteration 2: 32 conjunctions
Iteration 3: 130 conjunctions
Iteration 4: 520 conjunctions
Iteration 1: 8 conjunctions
Iteration 2: 32 conjunctions
Iteration 3: 120 conjunctions
Iteration 4: 640 conjunctions
Iteration 1: 8 conjunctions
Iteration 2: 40 conjunctions
Iteration 3: 220 conjunctions
Iteration 4: 428 conjunctions
Iteration 1: 8 conjunctions
Iteration 2: 44 conjunctions
Iteration 3: 124 conjunctions
Iteration 4: 808 conjunctions
Iteration 1: 8 conjunctions
Iteration 2: 43 conjunctions
Iteration 3: 301 conjunctions
Iteration 4: 1114 conjunctions
Iteration 1: 8 conjunctions
Iteration 2: 51 conjunctions
Iteration 3: 194 conjunctions
Iteration 4: 768 conjunctions
Iteration 1: 8 conjunctions
Iteration 2: 38 conjunctions
Iteration 3: 194 conjunctions
Iteration 4: 908 conjunctions
Iteration 1: 8 conjunctions
Iteration 2: 32 conjunctions
Iteration 3: 210 conjunctions
Iteration 4: 1528 conjunctions
Iteration 1: 8 conjunctions
Iteration 2: 60 conjunctions
Itera

KeyboardInterrupt: 

In [7]:
cs.get_conjunction_set_df()['label'].value_counts()

positive    7782
negative    7782
Name: label, dtype: int64

In [ ]:
conjunctionPredictions

In [ ]:
rf_predicions

In [ ]:
cs.label_names

In [ ]:
indx=np.argwhere(np.array(conjunctionPredictions!=rf_predicions)==True)[0][0]

In [ ]:
test_x[indx]

In [ ]:
{k:v for v,k in zip(test_x[indx],iris.feature_names)}

In [ ]:
b=cs.get_instance_branch(test_x[indx])
print(b.toString())
print(b.features_upper)

In [ ]:
rf.predict_proba(test_x[indx])

In [ ]:
tree=rf.estimators_[0].tree_
print(tree.value)
print(tree.n_node_samples)
tree.value[0][0][0]
tree?

In [ ]:
rf.predict_proba?

In [ ]:
test_y[1]

In [ ]:
for b in cs.conjunctionSet:
    if b.containsInstance(test_x[1]):
        print(b.toString())
        print(b.getLabel())

In [ ]:
rf.predict_proba(test_x[21])

In [ ]:
test_y[21]

In [ ]:
np.argwhere(np.array(rf_predicions!=conjunctionPredictions)==True)

In [ ]:
np.where?

In [ ]:
for b in mg.conjunctionSet:
    print(b.toString())
    print("")

In [ ]:
records=[]
for b in mg.conjunctionSet:
    records.extend(b.get_branch_records())


In [ ]:
data=pd.DataFrame(records).fillna(0)

In [ ]:
est=rf.estimators_[0]
t=est.tree_
export_graphviz(t,out_file='tree.dot',feature_names=iris.feature_names,class_names=iris.target_names,  
                         filled=True, rounded=True)